In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np

2025-01-03 12:25:00.263586: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-03 12:25:00.281143: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735907100.300812    7350 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735907100.306847    7350 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-03 12:25:00.326299: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [16]:
def create_model():
    base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(96, 96, 3)
    )
    
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(43, activation='softmax')(x)

    return Model(inputs=base_model.input, outputs=predictions)


In [44]:
def create_data_generators():
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=False,
        fill_mode='nearest',
        brightness_range=[0.8, 1.2],
    )

    val_datagen = ImageDataGenerator(
        rescale=1./255,
    )

    return train_datagen, val_datagen

In [45]:
def train_model(model, train_generator, validation_generator, epochs=50):
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=10,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=5,
            min_lr=1e-6
        )
    ]

    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=epochs,
        callbacks=callbacks
    )

    return history

In [46]:
model = create_model()

train_datagen, val_datagen = create_data_generators()

In [47]:
train_generator = train_datagen.flow_from_directory(
    './data/train',  
    target_size=(96, 96),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    './data/train',  
    target_size=(96, 96),
    batch_size=32,
    class_mode='categorical'
)


Found 39209 images belonging to 43 classes.
Found 39209 images belonging to 43 classes.


In [ ]:
history = train_model(model, train_generator=train_generator, validation_generator=validation_generator)

In [2]:
import os
from PIL import Image
from tensorflow import keras

In [3]:
signs = 43

In [76]:
def resize_images(size=(32, 32)):
    for i in range(signs):
        input_directory = f"./data/train/{i}"
        output_directory = f"./data/train-r/{i}"
        
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)
    
        for filename in os.listdir(input_directory):
            input_path = os.path.join(input_directory, filename)
            output_path = os.path.join(output_directory, filename)
    
            try:
                with Image.open(input_path) as img:
                    img = img.resize(size)
                    img.save(output_path)
                    print(f"Resized and saved: {output_path}")
            except Exception as e:
                print(f"Error processing {input_path}: {e}")

In [ ]:
resize_images(size=(32, 32))

In [4]:
def load_images():
    
    sign_images = []
    sign_indexes = []

    for i in range(signs):
        
        input_directory = f"./data/train-r/{i}"
        
        for filename in os.listdir(input_directory):
            input_path = os.path.join(input_directory, filename)

            try:
                with Image.open(input_path) as img:
                    sign_images.append(np.array(img))
                    sign_indexes.append(i)
            except Exception as e:
                print(f"Error processing {input_path}: {e}")

    sign_images = np.array(sign_images)
    sign_indexes = np.array(sign_indexes)

    return sign_images, sign_indexes

In [5]:
sign_images, sign_indexes = load_images()

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_val, y_train, y_val = train_test_split(sign_images, sign_indexes, test_size=0.2, random_state=42)

In [8]:
y_train = keras.utils.to_categorical(y_train, 43)
y_val = keras.utils.to_categorical(y_val,43)

In [9]:
def create_model():
    base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(32, 32, 3)
    )
    
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(43, activation='softmax')(x)

    return Model(inputs=base_model.input, outputs=predictions)

In [10]:
model = create_model()

checkpoint = tf.keras.callbacks.ModelCheckpoint (
    "./checkpoints/traffic_sign_classification_resnet50_{epoch:02d}_{val_accuracy:.3f}.keras",
    save_best_only=True,
    monitor="val_accuracy"
)

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)



I0000 00:00:1735907158.295593    7350 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:1e.0, compute capability: 7.5


In [11]:
history = model.fit(X_train,y_train,
                    validation_data=(X_val, y_val),
                    epochs=20, batch_size=32, 
                    callbacks=[checkpoint])

Epoch 1/20


I0000 00:00:1735907170.244325    7468 service.cc:148] XLA service 0x7fca780024c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1735907170.244349    7468 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2025-01-03 12:26:10.406356: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1735907171.546340    7468 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-01-03 12:26:12.172996: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[32,64,8,8]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,64,8,8]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_m

 24/981 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.0267 - loss: 8.0971     

I0000 00:00:1735907174.551736    7468 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


973/981 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1710 - loss: 3.9357

2025-01-03 12:26:22.625564: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[7,64,8,8]{3,2,1,0}, u8[0]{0}) custom-call(f32[7,64,8,8]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2025-01-03 12:26:22.725755: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[7,128,4,4]{3,2,1,0}, u8[0]{0}) custom-call(f32[7,128,4,4]{3,2,1,0}, f32[128,128,3,3]{3,2,1,0}, f32[128]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForwar

981/981 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1716 - loss: 3.9279

2025-01-03 12:26:30.582337: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[2,64,8,8]{3,2,1,0}, u8[0]{0}) custom-call(f32[2,64,8,8]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2025-01-03 12:26:30.641145: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[2,128,4,4]{3,2,1,0}, u8[0]{0}) custom-call(f32[2,128,4,4]{3,2,1,0}, f32[128,128,3,3]{3,2,1,0}, f32[128]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForwar

981/981 ━━━━━━━━━━━━━━━━━━━━ 28s 18ms/step - accuracy: 0.1717 - loss: 3.9269 - val_accuracy: 0.4745 - val_loss: 1.8310
Epoch 2/20
981/981 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.3968 - loss: 2.0485 - val_accuracy: 0.5738 - val_loss: 1.4056
Epoch 3/20
981/981 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.4873 - loss: 1.6484 - val_accuracy: 0.6421 - val_loss: 1.1723
Epoch 4/20
981/981 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.5525 - loss: 1.4032 - val_accuracy: 0.6947 - val_loss: 0.9938
Epoch 5/20
981/981 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.6038 - loss: 1.2265 - val_accuracy: 0.7269 - val_loss: 0.8886
Epoch 6/20
981/981 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.6423 - loss: 1.0905 - val_accuracy: 0.7585 - val_loss: 0.7842
Epoch 7/20
981/981 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.6882 - loss: 0.9629 - val_accuracy: 0.7784 - val_loss: 0.7000
Epoch 8/20
981/981 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.7174 - loss: 0.8612 - val_accuracy: 0.7997 - va